In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random

  Activating environment at `~/Documents/planar_quadruped_landing/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl")

WebIO._IJuliaInit()

reference_trajectory

## Reference Trajectory

## Problem Definition

In [3]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 0.6
dt = 0.02
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N)
t_trans = 0.2

n = 14
m = 4

# Initial Conditions
# currently, we assume the initial mode ID is 1
# xinit = [-0.4;0.6;-3.1415/9;        0.0;0.0; -1.0;0.2; 0.0;-0.5;0.0; 0.0;0.0; 0.0;-0.5]
# xterm = [-lb/2;sqrt(l1^2+l2^2);0.0; 0.0;0.0; -lb;0.0;  0.0;0.0;0.0;  0.0;0.0; 0.0;0.0]
xinit = zeros(n)
xinit[1] = -lb/2                  # xb
xinit[2] = sqrt(l1^2+l2^2) + 0.05 # yb
xinit[3] = 10 * pi / 180          # theta
xinit[6] = -lb                    # x2
xinit[7] = 0.05                   # y2
xinit[9] = -3.0                   # yb_dot
xinit[12] = -3.0                  # y2_dot

xterm = zeros(n)
xterm[1] = -lb/2           # xb
xterm[2] = sqrt(l1^2+l2^2) # yb
xterm[6] = -lb             # x2

init_mode = 1

# Reference Trajectory
Xref, Uref = reference_trajectory(model, times, t_trans, xinit, xterm, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([1.0;1.0;1.0; 1.0;1.0; 1.0;1.0; 1.0;1.0;1.0; 1.0;1.0; 1.0;1.0])
R = Diagonal(fill(1e-3,4))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);

## Solve

In [4]:
nlp.cinds

4-element Vector{UnitRange{Int64}}:
 1:14
 15:28
 29:448
 449:479

In [31]:
# Initial guess
xtransit = zeros(n)
xtransit[1] = -lb/2 # xb
xtransit[2] = l1    # yb
xtransit[3] = 0     # theta
xtransit[6] = -lb   # x2

# Random.seed!(1)
# Uguess = [u + 0.1*randn(length(u)) for u in Uref]
# Xguess = [x + 0.1*randn(length(x)) for x in Xref]

Uguess = [zeros(length(u)) for u in Uref]
Xguess = [zeros(length(x)) for x in Xref]

timesteps_phase_1 = nlp.k_trans

for i = 1:timesteps_phase_1
    Xguess[i] = xinit + (xtransit - xinit)/timesteps_phase_1 * i
end

# display((xtransit - xinit)/timesteps_phase_1)

In [32]:
Z0 = packZ(nlp, Xguess, Uguess);
# nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end], use_sparse_jacobian=false);

# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-2)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   265366
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      554
                     variables with only lower bounds:       31
                variables with lower and upper bounds:    

([-0.25000025029511064, 0.4035556567103768, 0.1745329255847195, -4.523987558400426e-8, 6.802784062858504e-7, -0.4999999942249929, 0.04999999710156418, -2.898410231543616e-9, -2.999999980956001, -4.802676090233731e-10  …  -1.9496065955950718e-10, -0.4999999872999998, -8.751628477147336e-9, -1.391129024022663e-8, 1.1726204188609845e-9, 8.588538294363627e-10, 2.0113251304822777e-9, -7.747645748683116e-11, 1.8835589808358483e-9, 3.132528838052696e-10], Ipopt.Optimizer)

In [33]:
Z_sol[1:14] - xinit

14-element Vector{Float64}:
 -2.5029511063756615e-7
  2.2661171030224914e-6
  3.8528655266212297e-10
 -4.523987558400426e-8
  6.802784062858504e-7
  5.775007083652639e-9
 -2.8984358246830766e-9
 -2.898410231543616e-9
  1.9043998822354524e-8
 -4.802676090233731e-10
 -2.569775308666538e-9
  1.6692227466919007e-7
  1.9417831550681523e-10
 -2.1894793044748974e-11

In [34]:
Z_sol[end-13:end] - xterm

14-element Vector{Float64}:
  5.402987274250037e-8
 -1.6087122745034321e-9
 -6.169913445426218e-10
 -2.809311668519319e-8
 -1.9496065955950718e-10
  1.2700000218135443e-8
 -8.751628477147336e-9
 -1.391129024022663e-8
  1.1726204188609845e-9
  8.588538294363627e-10
  2.0113251304822777e-9
 -7.747645748683116e-11
  1.8835589808358483e-9
  3.132528838052696e-10

EXIT: Restoration failed!

EXIT: Converged to a point of local infeasibility. Problem may be infeasible.

In [ ]:
display(Z_sol)
using DelimitedFiles
using CSV
writedlm( "data.csv",  Z_sol, ',')